In [1]:
from utils.schedule_utils import load_data_split,table_to_str,table_to_str_sql,find_intersection_and_add_row_id,Prepare_Data_for_Operator_Sequence,format_document,batch_rerank_scores,ROLLBACK,merge_clean_and_format_df_dict,retrieve_rows_by_subtables,process_error_analysis_list
import pickle
import numpy as np
import pandas as pd
import os
import json
from FlagEmbedding import FlagReranker
from tqdm import tqdm
from typing import Dict, Any
from utils.evaluator import Evaluator
from utils.prompt_generate import build_wikitq_prompt_from_df,evaluate_predictions,filter_dataframe_from_responses,fix_sql_query,match_subtables,retrieve_rows_by_subtables,build_tab_fact_prompt_from_df
from utils.async_llm import infer_prompts

/home/aizoo/miniconda3/envs/hstar/lib/python3.9/site-packages/torch/cuda/__init__.py:63: FutureWarning: The pynvml package is deprecated. Please install nvidia-ml-py instead. If you did not install pynvml directly, please report this to the maintainers of the package that installed pynvml for you.
  import pynvml  # type: ignore[import]


In [ ]:
## wikitq: 0.491 second/query 
## tab_fact: 0.29 second/query

In [2]:
from vllm import LLM, SamplingParams
from transformers import AutoTokenizer
import pandas as pd
import numpy as np
from time import sleep
import argparse
from vllm.lora.request import LoRARequest
import json
from tqdm import tqdm
import os 

INFO 10-26 23:14:51 [__init__.py:216] Automatically detected platform cuda.


In [22]:
llm_path = '/data/workspace/yanmy/models/Qwen3-4B-Instruct-2507'
# llm_path = '/data/workspace/yanmy/models/Qwen2.5-7B-Instruct/'
llm = LLM(model=llm_path,
        tensor_parallel_size=2,
        enforce_eager=True,
        enable_chunked_prefill=True,
        max_model_len=23000,
        # num_scheduler_steps=10,
        enable_prefix_caching=True,
        gpu_memory_utilization = 0.85,
        )

tokenizer = AutoTokenizer.from_pretrained(llm_path)

INFO 10-17 23:27:02 [utils.py:233] non-default args: {'max_model_len': 23000, 'tensor_parallel_size': 2, 'enable_prefix_caching': True, 'gpu_memory_utilization': 0.85, 'disable_log_stats': True, 'enforce_eager': True, 'enable_chunked_prefill': True, 'model': '/data/workspace/yanmy/models/Qwen3-4B-Instruct-2507'}
INFO 10-17 23:27:02 [model.py:547] Resolved architecture: Qwen3ForCausalLM


`torch_dtype` is deprecated! Use `dtype` instead!


INFO 10-17 23:27:02 [model.py:1510] Using max model len 23000
INFO 10-17 23:27:02 [scheduler.py:205] Chunked prefill is enabled with max_num_batched_tokens=8192.
INFO 10-17 23:27:02 [__init__.py:381] Cudagraph is disabled under eager mode
(EngineCore_DP0 pid=1542434) INFO 10-17 23:27:03 [core.py:644] Waiting for init message from front-end.
(EngineCore_DP0 pid=1542434) INFO 10-17 23:27:03 [core.py:77] Initializing a V1 LLM engine (v0.11.0) with config: model='/data/workspace/yanmy/models/Qwen3-4B-Instruct-2507', speculative_config=None, tokenizer='/data/workspace/yanmy/models/Qwen3-4B-Instruct-2507', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=23000, download_dir=None, load_format=auto, tensor_parallel_size=2, pipeline_parallel_size=1, data_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=True, kv_cache_dtype=auto, device_config=cuda, structured_outpu

Loading safetensors checkpoint shards:   0% Completed | 0/3 [00:00<?, ?it/s]


(EngineCore_DP0 pid=1542434) (Worker_TP1 pid=1542458) INFO 10-17 23:27:07 [default_loader.py:267] Loading weights took 0.87 seconds
(EngineCore_DP0 pid=1542434) (Worker_TP0 pid=1542456) INFO 10-17 23:27:08 [default_loader.py:267] Loading weights took 0.87 seconds
(EngineCore_DP0 pid=1542434) (Worker_TP0 pid=1542456) INFO 10-17 23:27:08 [gpu_model_runner.py:2653] Model loading took 3.8657 GiB and 0.997680 seconds
(EngineCore_DP0 pid=1542434) (Worker_TP1 pid=1542458) INFO 10-17 23:27:09 [gpu_model_runner.py:2653] Model loading took 3.8657 GiB and 0.996497 seconds
(EngineCore_DP0 pid=1542434) (Worker_TP0 pid=1542456) INFO 10-17 23:27:13 [gpu_worker.py:298] Available KV cache memory: 14.64 GiB
(EngineCore_DP0 pid=1542434) (Worker_TP1 pid=1542458) INFO 10-17 23:27:13 [gpu_worker.py:298] Available KV cache memory: 14.64 GiB
(EngineCore_DP0 pid=1542434) INFO 10-17 23:27:13 [kv_cache_utils.py:1087] GPU KV cache size: 213,248 tokens
(EngineCore_DP0 pid=1542434) INFO 10-17 23:27:13 [kv_cache_uti

In [23]:
def response_vllm(llm,all_instructions,sample_num: int):
    text_all = []
    # all_instructions = prompts_list
    for prompt in tqdm(all_instructions, desc="Formatting Prompts"):
        messages = [{"role": "user", "content": prompt}]
        text = tokenizer.apply_chat_template(
            messages,
            tokenize=False,
            add_generation_prompt=True
        )
        text_all.append(text)
    if sample_num == 1:
        sampling_params = SamplingParams(temperature=0, top_p=1, max_tokens=2048) 
    else:
        ## top_k/min_p 传不进去
        sampling_params = SamplingParams(n=sample_num, temperature=0.7, top_p=0.8, top_k=20, min_p=0, max_tokens=2048) 
    outputs = llm.generate(text_all, sampling_params)
    generation_list = []
    for output in tqdm(outputs, desc="Processing Outputs"):
        generated_text = [o.text for o in output.outputs]
        generation_list.append(generated_text)
    return generation_list


In [3]:
## 参数记录 for wikitq
N_PARALLEL = 32 ## only for convert_df_type_parallel.py of data preprocess，一次性离线的
embedding_base_model_path = '/data/workspace/yanmy/models/bge-m3'
router_model_path = '/data/workspace/yanmy/HybridRAG/H-STAR/router/bge-m3-finetuned/' ## fine-tuned router model path
check_moedl_path = '/data/workspace/yanmy/HybridRAG/H-STAR/check/output/bge-reranker-v2-m3-finetuned/' ## check model path
tmp_save_path = 'datasets/schedule_test/fetaqa' ## 临时存储路径
dataset_name = 'fetaqa'
split = 'test'
tau = 0.82
check_tau = 0.8
ALL_LABELS = [
    'Base', 'Select_Row', 'Select_Column', 'Execute_SQL', 'RAG_20_5', 
]


### Pipeline 的过程分为以下调度步骤
- 所有调度需要的文件均放在了 `schedule_test/datasets/wikitq`与`schedule_test/datasets/tab_fact`文件夹里面
1. （独立的）跑下面的指令：`python convert_df_type_parallel.py --dataset_name tab_fact --split test_small --output_path datasets/schedule_test/tab_fact/wikitq_df_processed.npy --num_workers 32`
2. (独立的)读取 `datasets/schedule_test/tab_fact/wikitq_df_processed.npy`,构造 inference_router 的文件，输出到`datasets/schedule_test/tab_fact/router_query.pkl`
3. (独立的)构造 DB
4. (GPU-21GB) inference router model，输出route result 到 `datasets/schedule_test/tab_fact/inference_result.pkl` 通过命令：`python inference_router.py --input_path datasets/schedule_test/tab_fact/router_query.pkl --model_path /data/workspace/yanmy/HybridRAG/H-STAR/router/bge-m3-router-tab_fact-hn --output_path datasets/schedule_test/tab_fact/inference_result.pkl`
5. (CPU) 解析 router 结果，生成初步的路径并开始任务调度(重排)
6. (GPU-1.8G) 根据 `datasets/schedule_test/tab_fact/RAG_index.npy`，执行 RAG 任务，根据下面的指令
7. (GPU-vLLM) LLM 根据初步路径，对`Select_Row`, `Select_Column`, `Execute_SQL`执行中间命令
8. (CPU) 根据中间结果，link db 做sql command parse，执行 sql,返回 subtable_list
9. (GPU-1.8G) 加载并运行 Check Model，跑 3 次（FORWARD 机制）
10. (GPU-vLLM) add execute SQL
11. (GPU-vLLM) generate qa command, inference vLLM, evaluate


In [4]:
## step:1
f'python convert_df_type_parallel.py --dataset_name {dataset_name} --split {split} --output_path {tmp_save_path}/wikitq_df_processed.npy --num_workers {N_PARALLEL}'

'python convert_df_type_parallel.py --dataset_name fetaqa --split test --output_path datasets/schedule_test/fetaqa/wikitq_df_processed.npy --num_workers 32'

In [12]:
## step 4: inference router model
f'python inference_router.py --input_path {tmp_save_path}/router_query.pkl --model_path {router_model_path} --output_path {tmp_save_path}/inference_result.pkl'

'python inference_router.py --input_path datasets/schedule_test/fetaqa/router_query.pkl --model_path /data/workspace/yanmy/HybridRAG/H-STAR/router/bge-m3-finetuned/ --output_path datasets/schedule_test/fetaqa/inference_result.pkl'

In [19]:
## step 6: RAG
f'python Hybrid_Retrieve.py --model_path {embedding_base_model_path} --dataset_name {dataset_name} --split {split} --index_path {tmp_save_path}/RAG_index.npy --output_path {tmp_save_path}/Hybrid_Retrieve_output.npy --max_rows 20 --max_cols 5 --processed_df_path {tmp_save_path}/wikitq_df_processed.npy'

'python Hybrid_Retrieve.py --model_path /data/workspace/yanmy/models/bge-m3 --dataset_name fetaqa --split test --index_path datasets/schedule_test/fetaqa/RAG_index.npy --output_path datasets/schedule_test/fetaqa/Hybrid_Retrieve_output.npy --max_rows 20 --max_cols 5 --processed_df_path datasets/schedule_test/fetaqa/wikitq_df_processed.npy'

In [9]:
### 读取必要文件（step 2）
dataset = load_data_split(dataset_name,split)
wikitq_df_processed = np.load(f'{tmp_save_path}/wikitq_df_processed.npy',allow_pickle=True).item()


In [10]:
## step 2: 构建semantic router的输入文件，并输出
semantic_router = {}
# semantic_router[
for index in range(len(dataset)):
    semantic_router[index] = {}
    semantic_router[index]['query'] = dataset[index]['question']
    semantic_router[index]['title'] = dataset[index]['table']['page_title']
    semantic_router[index]['table'] = wikitq_df_processed[index]
    label_list = []
    semantic_router[index]['label'] = label_list
with open(f'{tmp_save_path}/router_query.pkl','wb') as f:
    pickle.dump(semantic_router,f)

In [11]:
## step 3: Consturct DB
from utils.multi_db_v2 import NeuralDB, Executor
table_titles = [dataset[i]['table']['page_title'] for i in range(len(dataset))]
db = NeuralDB(tables=wikitq_df_processed, table_titles=table_titles)
executor = Executor()

Initializing NeuralDB...


Creating tables in DB:   0%|          | 0/2003 [00:00<?, ?it/s]

In [14]:
## step 5
with open(f'{tmp_save_path}/inference_result.pkl', 'rb') as f:
    error_analysis_row = pickle.load(f)
# with open(f'../router/inference_results.pkl', 'rb') as f:
#     error_analysis_row = pickle.load(f)
ranked_result = process_error_analysis_list(error_analysis_row, truncate=True, tau=0.82)
pd.DataFrame([str(r) for r in ranked_result.values()]).value_counts()


0                                             
['Execute_SQL']                                   1584
['Base']                                           130
['Select_Row', 'Select_Column', 'Execute_SQL']     125
['Select_Column', 'Select_Row', 'Execute_SQL']      62
['Select_Row', 'Execute_SQL']                       43
['RAG_20_5']                                        16
['Select_Row']                                      14
['RAG_20_5', 'Select_Row', 'Execute_SQL']            7
['Select_Column', 'Execute_SQL']                     6
['Select_Row', 'RAG_20_5', 'Execute_SQL']            5
['RAG_20_5', 'Execute_SQL']                          4
['Select_Column']                                    3
['Select_Column', 'RAG_20_5', 'Execute_SQL']         3
['RAG_20_5', 'Select_Column', 'Execute_SQL']         1
Name: count, dtype: int64

In [17]:
## step 5
## Organize LLM query list
LLM_query_list = {}
for method in ALL_LABELS:
    LLM_query_list[method] = {}
    LLM_query_list[method]['index'] = []
    LLM_query_list[method]['query'] = []
    LLM_query_list[method]['qa'] = []
for index in range(len(dataset)):
    for method in ALL_LABELS:
        if method in ranked_result[index]:
            # method_list = ranked_result[index][method]
            # method_list.append(index)
            LLM_query_list[method]['index'].extend([index])
            if method=='Select_Column':
                prompt = build_wikitq_prompt_from_df(dataset,wikitq_df_processed[index],index,template_path='../prompts/col_select_sql.txt',processed=True)
                LLM_query_list[method]['query'].extend([prompt])
            elif method=='Select_Row':
                prompt = build_wikitq_prompt_from_df(dataset,wikitq_df_processed[index],index,template_path='../prompts/row_select_sql.txt',processed=True)
                LLM_query_list[method]['query'].extend([prompt])
            elif method == 'Execute_SQL':
                prompt = build_wikitq_prompt_from_df(dataset,wikitq_df_processed[index],index,template_path='../prompts/sql_reason_fetaqa.txt',processed=True)
                LLM_query_list[method]['query'].extend([prompt])
            # elif method == 'Base':
            #     prompt = build_wikitq_prompt_from_df(dataset,wikitq_df_processed[index],index,template_path='../prompts/text_reason_tabfact.txt',processed=True)
            #     LLM_query_list[method]['qa'].extend([prompt])


In [18]:
## step 5 暂存 RAG 的 list，输出到路径
np.save(f'{tmp_save_path}/RAG_index.npy',LLM_query_list['RAG_20_5']['index'])

- 执行`Hybrid_Retrieve.py` 任务

In [20]:
## load Hybrid_Retrieve 的结果
RAG_20_5 = np.load(f'{tmp_save_path}/Hybrid_Retrieve_output.npy',allow_pickle=True).item()

In [ ]:
## step 7: api vLLM
### Query LLM with async engine，启动模型采用 sh llm_server.sh，这个是不是可以放在调度的第一个环节，不计入时间

metrics_rows = {}
summary = {}
for method in ['Select_Row','Select_Column']:
    prompt_list = LLM_query_list[method]['query']
    response_list,metrics_rows[method],summary[method] = infer_prompts(prompt_list, temperature=0.7, top_p=0.8, sample_num=2)
    LLM_query_list[method]['response'] = response_list
for method in ['Execute_SQL']:
    prompt_list = LLM_query_list[method]['query']
    response_list,metrics_rows[method],summary[method] = infer_prompts(prompt_list, temperature=0.7, top_p=0.8, sample_num=3)
    LLM_query_list[method]['response'] = response_list

In [21]:
## step 7
### Query LLM with async engine，启动模型采用 sh llm_server.sh，这个是不是可以放在调度的第一个环节，不计入时间

metrics_rows = {}
summary = {}
for method in ['Select_Row','Select_Column']:
    prompt_list = LLM_query_list[method]['query']
    response_list = response_vllm(llm,prompt_list, sample_num=2)
    LLM_query_list[method]['response'] = response_list
for method in ['Execute_SQL']:
    prompt_list = LLM_query_list[method]['query']
    response_list = response_vllm(llm,prompt_list, sample_num=3)
    LLM_query_list[method]['response'] = response_list

NameError: name 'response_vllm' is not defined

In [25]:
## step: 7 offline Batch
all_list = []
index_list = {}
index_count = 0
for method in ['Select_Row','Select_Column']:
    prompt_list = LLM_query_list[method]['query']
    all_list.extend(prompt_list)
    index_count += len(prompt_list)
    index_list[method] = index_count
    # response_list = response_vllm(prompt_list, sample_num=2)
    # LLM_query_list[method]['response'] = response_list
for method in ['Execute_SQL']:
    prompt_list = LLM_query_list[method]['query']
    all_list.extend(prompt_list)
    index_count += len(prompt_list)
    index_list[method] = index_count
    # response_list = response_vllm(prompt_list, sample_num=2)
    # LLM_query_list[method]['response'] = response_list
response_list = response_vllm(llm, all_list, sample_num=3)
# for method in ['Select_Row','Select_Column','Execute_SQL']:
    # response_subset = 
LLM_query_list['Select_Row']['response'] = response_list[0:index_list['Select_Row']]
LLM_query_list['Select_Column']['response'] = response_list[index_list['Select_Row']:index_list['Select_Column']]
LLM_query_list['Execute_SQL']['response'] = response_list[index_list['Select_Column']:]

Formatting Prompts:   0%|          | 0/2296 [00:00<?, ?it/s]

Formatting Prompts: 100%|██████████| 2296/2296 [00:00<00:00, 30328.23it/s]


Adding requests:   0%|          | 0/2296 [00:00<?, ?it/s]

Processed prompts:   0%|          | 0/6888 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s…

Processing Outputs: 100%|██████████| 2296/2296 [00:00<00:00, 154254.72it/s]


- row_sql

In [28]:
## step 8: SQP parse row

from tqdm.notebook import tqdm
sub_table_list_all = {}
filtered_tables_row = {}
row_sql_index_list = LLM_query_list['Select_Row']['index']
row_sql_response_list = LLM_query_list['Select_Row']['response']
for i in range(len(row_sql_index_list)):
    # sample_num = [0,1]
    sample_num = [0,1,2]
    sub_table_list = []
    for sample_index in sample_num:
        ## 相对距离
        index = row_sql_index_list[i]
        #  = row_sql_response_list[i][sample_index]
        # relevant_index = row_sql_index_list.index(index)
        original_text = row_sql_response_list[i][sample_index]
        sql = fix_sql_query(response_text=original_text,table_df=wikitq_df_processed[index],table_title=table_titles[index])
        try:
            result_1 = executor.sql_exec(sql.replace('``','`').replace("COUNT(*)", "*"), db, table_id=index)
            sub_table_list.append(pd.DataFrame(result_1['rows'],columns=result_1['header']))
        except:
            continue
    # print(dataset[index]['question'],dataset[index]['answer_text'])
    # match_subtables(large_table=wikitq_df_processed[index], sub_tables=[pd.DataFrame(result_1['rows'],columns=result_1['header'])])
    # pd.DataFrame(result_1['rows'],columns=result_1['header'])
    sub_table_list_all[index] = sub_table_list
    filtered_df = retrieve_rows_by_subtables(wikitq_df_processed[index],sub_table_list)
    if len(filtered_df) == 0:
        filtered_df = wikitq_df_processed[index]
    filtered_tables_row[index] = filtered_df

Executing on table_id 8 ('w8'): SELECT role FROM w WHERE year = '2007';
Executing on table_id 8 ('w8'): SELECT role FROM w WHERE year = '2007';
Executing on table_id 8 ('w8'): SELECT role FROM w WHERE year = '2007';
Executing on table_id 11 ('w11'): SELECT `club performance_2` FROM w WHERE `club performance` LIKE '2002%';
Executing on table_id 11 ('w11'): SELECT `club performance_2` FROM w WHERE `club performance` LIKE '2002%';
Executing on table_id 11 ('w11'): -Decompose:   - Part 1: Find the row where the year is 2002     Response 1: SELECT * FROM w WHERE `club performance` LIKE '2002%';   - Part 2: Get the club name (second column) from that row     Response 2: SELECT `club performance_2` FROM w WHERE `club performance` LIKE '2002%';  -Retrieval:   SQL: SELECT `club performance_2` FROM w WHERE `club performance` LIKE '2002%';
Executing on table_id 16 ('w16'): - Decompose:   - Part 1: Find the second album in chronological order     Response 1: SELECT `album`, `year` FROM w ORDER BY 

- col_sql

In [29]:
## step 8: SQP parse column
filtered_tables = {}
filtered_headers = {}
col_sql_index_list = LLM_query_list['Select_Column']['index']
col_sql_response_list = LLM_query_list['Select_Column']['response']
# for ind in tqdm(range(len(wikitq_df_processed))):
for i in range(len(col_sql_index_list)):
    ind = col_sql_index_list[i]
    
    input_df = wikitq_df_processed[ind]
    response_list = col_sql_response_list[i]
    assert isinstance(response_list,list)
    filtered_table, final_headers = filter_dataframe_from_responses(response_list, input_df, add_row_id=True)
    filtered_tables[ind] = filtered_table
    filtered_headers[ind] = final_headers

- SQL-exec

In [30]:
## step 8: SQP parse execute
from tqdm.notebook import tqdm
sample_num = 3
sql_exec_df = {}
valid_parse = 0
warning_list = []
sql_executable_count = []
exec_sql_index_list = LLM_query_list['Execute_SQL']['index']
exec_sql_response_list = LLM_query_list['Execute_SQL']['response']
# for index in tqdm(range(len(wikitq_df_processed))):
for i in range(len(exec_sql_index_list)):
    index = exec_sql_index_list[i]
    sql_exec_df[index] = []
    for sample_ind in range(sample_num):
        
        # original_text = eval(wikitq_df_output.iloc[index,-1])[sample_ind]
        original_text = exec_sql_response_list[i][sample_ind]
        sql = fix_sql_query(response_text=original_text,table_df=wikitq_df_processed[index],table_title=table_titles[index])
        if [index,sample_ind] in warning_list:
            sql = ''
            print(index,sample_ind)
        if sql!='':
            # print(result)
            try:
                result = executor.sql_exec(sql.replace('``','`'), db, table_id=index,add_row_id = True)
                df = pd.DataFrame(result['rows'],columns=result['header'])
            except:
                df = pd.DataFrame()
        else:
            df = pd.DataFrame()
        sql_exec_df[index].append(df)
        if(len(df)>0):
            valid_parse+=1
            sql_pair = {}
            sql_pair['id'] = index
            sql_pair['sample_ind'] = sample_ind
            sql_pair['sql'] = sql
            sql_pair['table'] = df
            sql_executable_count.append(sql_pair)
sql_exec_df_output = merge_clean_and_format_df_dict(sql_exec_df)
    # print(original_text)

Executing on table_id 0 ('w0'): SELECT "single" FROM w WHERE "year" = '1988' AND "album" = 'Across the Rio Grande';
Executing on table_id 0 ('w0'): SELECT "single" FROM w WHERE "year" = '1988' AND "album" = 'Across the Rio Grande';
Executing on table_id 0 ('w0'): SELECT "single" FROM w WHERE "year" = '1988' AND "album" = 'Across the Rio Grande';
Executing on table_id 1 ('w1'): SELECT "fate" FROM w ORDER BY "tonnage (grt)" DESC LIMIT 2;
Executing on table_id 1 ('w1'): SELECT "fate" FROM w ORDER BY "tonnage (grt)" DESC LIMIT 2;
Executing on table_id 1 ('w1'): SELECT "fate" FROM w ORDER BY "tonnage (grt)" DESC LIMIT 2;
Executing on table_id 4 ('w4'): SELECT CASE WHEN COUNT(*) > 0 THEN 'yes' ELSE 'no' END AS answer FROM w WHERE "date" LIKE '2012%' AND ("arena" IS NULL OR "arena" = '');
Executing on table_id 4 ('w4'): SELECT COUNT(*) FROM w WHERE "date" LIKE '2012%' AND "arena" IS NULL OR "arena" = '';
Executing on table_id 4 ('w4'): SELECT COUNT(*) FROM w WHERE SUBSTR("date", 1, 4) = '2012

/data/workspace/yanmy/HybridRAG/H-STAR/utils/schedule_utils.py:276: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  final_df = unique_df.fillna('')
/data/workspace/yanmy/HybridRAG/H-STAR/utils/schedule_utils.py:276: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  final_df = unique_df.fillna('')
/data/workspace/yanmy/HybridRAG/H-STAR/utils/schedule_utils.py:276: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future beha

In [31]:
## step 8: SQP parse aggregate
processed_table = {} ## processed_table包括了inference LLM后所有结果的聚合,key 与 method_key 保持一致
processed_table['Base'] = wikitq_df_processed
processed_table['Select_Row'] = filtered_tables_row
processed_table['Select_Column'] = filtered_tables
processed_table['RAG_20_5'] = RAG_20_5
processed_table['Execute_SQL'] = sql_exec_df_output ## SQL聚合结果！
processed_table['Execute_SQL_count'] = sql_executable_count ## SQL单个结果，用作 CrossEncoder 跨模态查询

In [32]:
## step 8: SQP parse aggregate
np.save(f'{tmp_save_path}/processed_table.npy',processed_table)

- Run Check Model

In [33]:
## Step 9: generated check model data 
# 对每一条数据，维护一个 Sequence_Operator
# 每条数据应该对应一个 Dict,这个Dict 包含了3 个参数：
    # id: index
    # Sequence: Operator Sequence
    # Terminated: True/False
    # Check Score: CrossEncoder 分数
    # Check Status: True/False,当前是否被计算
# 对每条数据，如果 Terminated = True，则不参与计算
# 对应的，如果 Terminated = False，那么查看 Check Status,如果 Statue=False，那么计算 Check_Score
# 每一轮计算之后，Check_Score 更新，Check Statue更新，该轮开始判定，用一个 threshold
# 如果 Check Status = True, Check Score >= Threshold，那么标注 Terminated = True，当前路径被接收
# 如果 Check Status = True, Check Score < Threshold，那么Terminated = False, Check Status = False, Check Score 清空
# Check Score 触发 RollBack 机制，即除了‘Execute_SQL'外，Extract 回退，重新计算当前流程
# 如果当前操作符只有'Execute_SQL’和空（所有 RollBack 均未被接收），那么触发 FORWARD,Terminated  = True，强制修改 Operator Sequence 为['Execute_SQL']
Check_Model_Data_Sequence = {}
for key in ranked_result.keys(): ## 遍历所有数据,初始化
    start_sequence = ranked_result[key]
    Check_Model_Data_Sequence[key] = {}
    Check_Model_Data_Sequence[key]['id'] = key
    Check_Model_Data_Sequence[key]['Sequence'] = start_sequence
    if start_sequence == ['Base'] or start_sequence == ['Execute_SQL']: ## Terminated 不参与计算
        Check_Model_Data_Sequence[key]['Terminated'] = True
        Check_Model_Data_Sequence[key]['Check_Status'] = False
        Check_Model_Data_Sequence[key]['Check_Score'] = 0.0
    else:
        Check_Model_Data_Sequence[key]['Terminated'] = False
        Check_Model_Data_Sequence[key]['Check_Status'] = False
        Check_Model_Data_Sequence[key]['Check_Score'] = 0.0
# Check_Model_Data_Sequence[4]
for key in Check_Model_Data_Sequence.keys():
    data_entry = Prepare_Data_for_Operator_Sequence(key,Check_Model_Data_Sequence[key]['Sequence'],dataset, processed_table)
    Check_Model_Data_Sequence[key]['data_entry'] = data_entry
## Check
    

- load ReRanking Model

In [34]:
## step 9 load rerank model
reranker_model = FlagReranker(check_moedl_path, use_fp16=True)

In [35]:
## step 9: iterative check
check_tau = check_tau
for loop in range(3):
    updated_data = batch_rerank_scores(reranker_model, Check_Model_Data_Sequence, batch_size=16)
    Check_Model_Data_Sequence = updated_data
    ## Check Terminal Status
    for key in Check_Model_Data_Sequence.keys():
        if Check_Model_Data_Sequence[key]['Terminated'] == True:
            continue
        else:
            if Check_Model_Data_Sequence[key]['Check_Status'] == True:
                if Check_Model_Data_Sequence[key]['Check_Score'] >= check_tau: ## 0.5 threshold
                    Check_Model_Data_Sequence[key]['Terminated'] = True
                else:
                    Check_Model_Data_Sequence[key]['Terminated'] = False
                    Check_Model_Data_Sequence[key]['Check_Status'] = False
                    Check_Model_Data_Sequence[key]['Check_Score'] = 0.0
                    current_sequence = Check_Model_Data_Sequence[key]['Sequence']
                    ROLLBACK_seq, terminated_flag = ROLLBACK(current_sequence)
                    Check_Model_Data_Sequence[key]['Sequence'] = ROLLBACK_seq
                    Check_Model_Data_Sequence[key]['Terminated'] = terminated_flag
                    if terminated_flag == False:
                        data_entry = Prepare_Data_for_Operator_Sequence(key,ROLLBACK_seq,dataset, processed_table)
                        Check_Model_Data_Sequence[key]['data_entry'] = data_entry
            else:
                raise ValueError("Check Status should be True when Terminated is False after reranking.")

Filtering items to rerank (Terminated=False and data_entry exists)...
Computing scores for 289 items with batch size 16...


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
initial target device:   0%|          | 0/2 [00:00<?, ?it/s]huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
initial target device:  50%|█████     | 1/2 [00:03<00:03,  3.56s/it]huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before th

Updating scores in the original dictionary...
Reranking complete.
Filtering items to rerank (Terminated=False and data_entry exists)...
Computing scores for 60 items with batch size 16...


Chunks: 100%|██████████| 2/2 [00:00<00:00, 16.07it/s]

Updating scores in the original dictionary...
Reranking complete.
Filtering items to rerank (Terminated=False and data_entry exists)...
No items to process.


### Check Additional Execute SQL and Execute

In [36]:
## step 10: add missing SQL
SQL_list_final = []
for index in range(len(dataset)):
    sequence = Check_Model_Data_Sequence[index]['Sequence']
    if sequence==[] or sequence.__contains__('Execute_SQL'):
        SQL_list_final.append(index)
add_sql_list = list(set(SQL_list_final) - set(LLM_query_list['Execute_SQL']['index']))
add_sql_query_list = []
for index in add_sql_list:
    prompt = build_wikitq_prompt_from_df(dataset,wikitq_df_processed[index],index,template_path='../prompts/sql_reason_wtq.txt',processed=True)
    add_sql_query_list.extend([prompt])
add_sql_response_list = response_vllm(llm,add_sql_query_list, sample_num=3)

Formatting Prompts:   0%|          | 0/4 [00:00<?, ?it/s]

Adding requests:   0%|          | 0/4 [00:00<?, ?it/s]

Processed prompts:   0%|          | 0/12 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

Processing Outputs:   0%|          | 0/4 [00:00<?, ?it/s]

In [79]:
## step 10: add missing SQL
SQL_list_final = []
for index in range(len(dataset)):
    sequence = Check_Model_Data_Sequence[index]['Sequence']
    if sequence==[] or sequence.__contains__('Execute_SQL'):
        SQL_list_final.append(index)
add_sql_list = set([i for i in range(len(dataset))]) - set(LLM_query_list['Execute_SQL']['index'])
add_sql_query_list = []
for index in add_sql_list:
    prompt = build_wikitq_prompt_from_df(dataset,wikitq_df_processed[index],index,template_path='../prompts/sql_reason_fetaqa.txt',processed=True)
    add_sql_query_list.extend([prompt])
add_sql_response_list = response_vllm(llm,add_sql_query_list, sample_num=3)

Formatting Prompts:   0%|          | 0/163 [00:00<?, ?it/s]

Adding requests:   0%|          | 0/163 [00:00<?, ?it/s]

Processed prompts:   0%|          | 0/489 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

Processing Outputs:   0%|          | 0/163 [00:00<?, ?it/s]

In [82]:
### step 10: add sql parse and execute
from tqdm.notebook import tqdm
sample_num = 3
sql_exec_df = {}
valid_parse = 0
warning_list = []
sql_executable_count = []
# exec_sql_index_list = LLM_query_list['Execute_SQL']['index']
# exec_sql_response_list = LLM_query_list['Execute_SQL']['response']
# for index in tqdm(range(len(wikitq_df_processed))):
add_sql_list = list(add_sql_list)
for i in range(len(add_sql_list)):
    index = add_sql_list[i]
    sql_exec_df[index] = []
    for sample_ind in range(sample_num):
        
        # original_text = eval(wikitq_df_output.iloc[index,-1])[sample_ind]
        original_text = add_sql_response_list[i][sample_ind]
        sql = fix_sql_query(response_text=original_text,table_df=wikitq_df_processed[index],table_title=table_titles[index])
        if [index,sample_ind] in warning_list:
            sql = ''
            print(index,sample_ind)
        if sql!='':
            # print(result)
            try:
                result = executor.sql_exec(sql.replace('``','`'), db, table_id=index,add_row_id = True)
                df = pd.DataFrame(result['rows'],columns=result['header'])
            except:
                df = pd.DataFrame()
        else:
            df = pd.DataFrame()
        sql_exec_df[index].append(df)
        if(len(df)>0):
            valid_parse+=1
            sql_pair = {}
            sql_pair['id'] = index
            sql_pair['sample_ind'] = sample_ind
            sql_pair['sql'] = sql
            sql_pair['table'] = df
            sql_executable_count.append(sql_pair)
sql_exec_df_output_new = merge_clean_and_format_df_dict(sql_exec_df)
    # print(original_text)
for index in sql_exec_df_output_new.keys():
    sql_exec_df_output[index] = sql_exec_df_output_new[index]
processed_table['Execute_SQL'] = sql_exec_df_output

Executing on table_id 3 ('w3'): SELECT COUNT(*) FROM t WHERE "album" = 'holly dunn';
Executing on table_id 515 ('w515'): SELECT "date", "country" FROM w WHERE "format" = 'digital download';
Executing on table_id 515 ('w515'): SELECT "date", "country" FROM w WHERE "format" = 'digital download';
Executing on table_id 515 ('w515'): SELECT "date", "country" FROM w WHERE "format" = 'digital download';
Executing on table_id 1541 ('w1541'): Select the "date" and "label" from that row.    Since this is a direct lookup and does not require counting, summing, or complex filtering, no SQL query is needed beyond a simple selection.  However, note that the table name is w and the column names are correctly referenced. The query will retrieve the required date and label.  Output: None;
Executing on table_id 1541 ('w1541'): Select the "date" and "label" from that row.  Since this is a direct lookup and does not involve counting, summing, or complex filtering, no SQL query is needed beyond a simple se

/data/workspace/yanmy/HybridRAG/H-STAR/utils/schedule_utils.py:276: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  final_df = unique_df.fillna('')
/data/workspace/yanmy/HybridRAG/H-STAR/utils/schedule_utils.py:276: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  final_df = unique_df.fillna('')
/data/workspace/yanmy/HybridRAG/H-STAR/utils/schedule_utils.py:276: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future beha

In [83]:
np.save(f'{tmp_save_path}/Check_Model_Data_Sequence.npy',Check_Model_Data_Sequence)

In [84]:
## step 8: SQP parse aggregate
np.save(f'{tmp_save_path}/processed_table_forward.npy',processed_table)

In [85]:
Check_Model_Data_Sequence = np.load(f'{tmp_save_path}/Check_Model_Data_Sequence.npy',allow_pickle=True).item()

In [41]:
## step 11: generate QA result
prompt_list = []
for index in range(len(dataset)):
    sequence = Check_Model_Data_Sequence[index]['Sequence']
    prompt = build_wikitq_prompt_from_df(dataset,Check_Model_Data_Sequence[index]['data_entry']['table'],index,template_path='../prompts/text_reason_fetaqa.txt',processed=True)
    if sequence==[] or sequence.__contains__('Execute_SQL'):
        # try:
        evidence = table_to_str_sql(processed_table['Execute_SQL'][index])
        prompt = prompt + evidence
    prompt_list.append(prompt)
# wikitq_df['instruction'] = prompt_list

In [89]:
## step 11: generate QA result
prompt_list = []
for index in range(len(dataset)):
    sequence = Check_Model_Data_Sequence[index]['Sequence']
    # prompt = build_wikitq_prompt_from_df(dataset,wikitq_df_processed[index],index,template_path='../prompts/text_reason_fetaqa.txt',processed=True)
    prompt = build_wikitq_prompt_from_df(dataset,wikitq_df_processed[index],index,template_path='../prompts/text_reason_fetaqa.txt',processed=True)
    # if sequence==[] or sequence.__contains__('Execute_SQL'):
        # try:
    evidence = table_to_str_sql(processed_table['Execute_SQL'][index])
    prompt = prompt + evidence
    prompt_list.append(prompt)
# wikitq_df['instruction'] = prompt_list

In [90]:
## step 11: execute QA
qa_final = response_vllm(llm,prompt_list, sample_num=1)
wikitq_df = pd.DataFrame(dataset)
wikitq_df['instruction'] = prompt_list
wikitq_df['predict'] = qa_final

Formatting Prompts:   0%|          | 0/2003 [00:00<?, ?it/s]

Adding requests:   0%|          | 0/2003 [00:00<?, ?it/s]

Processed prompts:   0%|          | 0/2003 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s…

Processing Outputs:   0%|          | 0/2003 [00:00<?, ?it/s]

In [91]:
import numpy as np
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from rouge_score import rouge_scorer

In [98]:


def calculate_metrics(predictions, ground_truths):
    """
    计算给定预测和真实标签的 BLEU 和 ROUGE 分数。

    参数:
    predictions (list of str): 模型生成的句子列表。
    ground_truths (list of str): 对应的标准参考句子列表。

    返回:
    dict: 包含平均分数的字典，键为 'bleu', 'rouge1', 'rouge2', 'rougeL'。
    """
    if len(predictions) != len(ground_truths):
        raise ValueError("预测列表和真实标签列表的长度必须相同。")

    # --- 初始化 ROUGE 评估器 ---
    # use_stemmer=True 表示在计算时使用词干提取
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    
    # --- 初始化用于存储每个样本分数的列表 ---
    bleu_scores = []
    rouge1_f_scores = []
    rouge2_f_scores = []
    rougeL_f_scores = []
    
    # --- NLTK 的 BLEU 平滑函数，避免因 n-gram 不匹配而导致分数为0 ---
    smoothie = SmoothingFunction().method1

    # --- 遍历每一对预测和真实标签 ---
    for pred, truth in tqdm(zip(predictions, ground_truths)):
        
        # --- 计算 ROUGE 分数 ---
        # scorer.score 返回一个包含 precision, recall, f-measure 的字典
        try:
            rouge_scores = scorer.score(truth, pred)
        except:
            # print(truth,pred)
            
            print(truth)
            print(f'fail to parse {pred}\ntruth: {truth}\n')
            pred = 'unknown answer'
            truth = 'unknown answer'
        # 我们通常关心 F1-score (fmeasure)
        rouge1_f_scores.append(rouge_scores['rouge1'].fmeasure)
        rouge2_f_scores.append(rouge_scores['rouge2'].fmeasure)
        rougeL_f_scores.append(rouge_scores['rougeL'].fmeasure)
        
        # --- 计算 BLEU 分数 ---
        # NLTK 的 sentence_bleu 需要将句子分词
        # 参考答案(truth)需要是一个列表的列表，因为一个预测可能对应多个参考答案
        truth_tokens = truth.split()
        pred_tokens = pred.split()
        
        bleu_score = sentence_bleu(
            truth_tokens, 
            pred_tokens, 
            smoothing_function=smoothie
        )
        bleu_scores.append(bleu_score)

    # --- 计算所有样本的平均分数 ---
    avg_bleu = np.mean(bleu_scores)
    avg_rouge1 = np.mean(rouge1_f_scores)
    avg_rouge2 = np.mean(rouge2_f_scores)
    avg_rougeL = np.mean(rougeL_f_scores)

    return {
        'bleu': avg_bleu,
        'rouge1': avg_rouge1,
        'rouge2': avg_rouge2,
        'rougeL': avg_rougeL
    }

# metrics = calculate_metrics([extract_answer_direct(q[0]) for q in qa_final], wikitq_df['answer'].tolist())
metrics = calculate_metrics([extract_answer_direct(q[0]) for q in qa_final], wikitq_df['answer'].tolist())
metrics

0it [00:00, ?it/s]

{'bleu': np.float64(0.0025289414062992124),
 'rouge1': np.float64(0.5651646873933305),
 'rouge2': np.float64(0.35309686469671303),
 'rougeL': np.float64(0.46546035445178374)}

In [97]:
extract_answer_direct(qa_final[55][0]).replace("  \n```",'')

'On March 9, 2008, Tyler Christopher achieved a new record (NR) in the 400 meters (indoor) event with a time of 45.67 seconds in Valencia, Spain.'

In [76]:
calculate_metrics([extract_answer_direct(qa_final[i][0]) for i in [7]], [dataset[i]['answer'] for i in [7]])

0it [00:00, ?it/s]

{'bleu': np.float64(0.0),
 'rouge1': np.float64(0.5128205128205129),
 'rouge2': np.float64(0.27027027027027023),
 'rougeL': np.float64(0.41025641025641024)}

In [77]:
def extract_answer(qa_final, index):
    qa_final_result = qa_final[index][0]
    split_1 = qa_final_result.split('Final Answer:')
    split_2 = split_1[-1].split('Output:')
    # print(len(split_1),len(split_2))
    if len(split_2)>1:
        answer = split_2[-1].strip()
    elif len(split_1)>1:
        answer = split_1[-1].strip()
    else:
        answer = qa_final_result.strip()
    return answer
def extract_answer_direct(input):
    qa_final_result = input
    split_1 = qa_final_result.split('Final Answer:')
    split_2 = split_1[-1].split('Output:')
    # print(len(split_1),len(split_2))
    if len(split_2)>1:
        answer = split_2[-1].strip()
    elif len(split_1)>1:
        answer = split_1[-1].strip()
    else:
        answer = input.strip()
    return answer
print(extract_answer_direct(qa_final[7][0]))
wikitq_df['answer'].tolist()[7]

Robert Boulter played two roles in 2008: Jimmy Dearden in "waking the dead" and Neil in "survivors".


'In 2008, Boulter starred as the character "Jimmy Dearden" in the two-part episode "Wounds" of the television series Waking the Dead.'

AttributeError: 'list' object has no attribute 'split'

In [45]:
metrics

{'bleu': np.float64(0.0008851297477959677),
 'rouge1': np.float64(0.2089121834369353),
 'rouge2': np.float64(0.1194370606974515),
 'rougeL': np.float64(0.17580847708541528)}